<a href="https://www.kaggle.com/code/aleksandrmorozov123/bitcoin-python-programming?scriptVersionId=96020860" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Exercises to understand bitcoin from the basics**

**First step - finite fields (mathematics)**

In [53]:
class FieldElement:
    def __init__ (self, num, prime):
        if num >= prime or num < 0 :
            error = 'Num {} not in the field range 0 to {}'.format (num, prime - 1)
            raise ValueError (error)
        self.num = num
        self.prime = prime

    def __repr__ (self):
        return 'FieldElement_{}({})'.format (self.prime, self.num)
    
    def __eq__ (self, other):
        if other is None:
            return False
        return self.num == other.num and self.prime == other.prime
    
    def __add__ (self, other):
        if self.prime != other.prime:
            raise TypeError ('Cannot add two numbers in different Fields')
        num = (self.num + other.num) % self.prime
        return self.__class__ (num, self.prime)
    
    def __pow__ (self, exponent):
        num = (self.num ** exponent) % self.prime
        return self.__class__ (num, self.prime)
        n = exponent
        while n < 0:
            n += self.prime - 1
        num = pow (self.num, n, self.prime)
        return self.__class__ (num, self.prime)
        n = exponent % (self.prime - 1)
        num = pow (self.num, n, self.prime)
        return self.__class__ (num, self.prime)
    
    def __ne__ (self, other):
        return not (self == other)
    
    def __mul__ (self, other):
        if self.prime != other.prime:
            raise TypeError ("Cannot multiply two numbers in different Fields")
        num = (self.num * other.num) % self.prime
        return self.__class__ (num, self.prime)
    
    def __truediv__ (self, other):
        if self.prime != other.prime:
            raise TypeError ('Cannot divide two numbers in different Fields')
        num = self.num * pow (other.num, self.prime - 2, self.prime) % self.prime
        return self.__class__ (num, self.prime)

In [3]:
a = FieldElement (5, 19)
b = FieldElement (8, 25)
print (a == b)

False


In [4]:
print (a == a)

True


In [5]:
a = FieldElement (5, 19)
b = FieldElement (13, 19)
c = FieldElement (6, 19)
print(a + b == c)

False


In [6]:
a = FieldElement (12, 15)
b = FieldElement (5, 16)
print (a ** 3 == b)

False


In [7]:
a = FieldElement (8, 19)
b = FieldElement (4, 12)
print (a ** - 5 == b)

False


**Elliptic curves**

In [68]:
class Point:
    def __init__ (self, x, y, a, b):
        self.a = a
        self.b = b
        self.x = x
        self.y = y
        if self.x is None and self.y is None:
            return
        if self.y ** 2 != self.x ** 3 + a * x + b:
            raise ValueError ('({}, {}) is not on the curve'.format (x, y))
            
    def __eq__ (self, other):
        return self.x == other.x and self.y == other.y \
            and self.a == other.a and self.b == other.b
        
        if self.y ** 2 != self.x ** 3 + a * x + b:
            raise ValueError ('({}, {}) is not on the curve'.format (x, y))
        
    def __add__ (self, other):
        if self.a != other.a or self.b != other.b:
            raise TypeError ('Points {}, {} are not on same curve'.format (self, other))
        if self.x is None:
            return other
        if other.x is None:
            return self
        if self.x == other.x and self.y != other.y:
            return self.__class__ (None, None, self.a, self.b)
        
    def __rmul__ (self, coefficient):
        product = self.__class__ (None, None, self.a, self.b)
        for _ in range (coefficient):
            product += self
        return product
        coef = coefficient
        current = self
        result = self.__class__ (None, None, self.a, self.b)
        while coef:
            if coef & 1:
                result += current
            current += current
            coef >>= 1
        return result
    
    def __ne__ (self, other):
        return not (self == other)
        

In [9]:
p1 = Point (-1, -1, 3, 5)
p2 = Point (-1, 1, 3, 5)
inf = Point (None, None, 3, 5)
print (p1 + inf)

In [10]:
print (inf + p2)

In [11]:
print (p1 + p2)

None


**Elliptic curve over Finite Fields**

In [55]:
a = FieldElement (num = 0, prime = 223)
b = FieldElement (num = 7, prime = 223)
x = FieldElement (num = 192, prime = 223)
y = FieldElement (num = 105, prime = 223)
p1 = Point (x, y, a, b)
print (p1)

**Point addition over Finite Fields**

**Scalar multiplicationn Redux**

In [54]:
prime = 223
a = FieldElement (num = 0, prime = prime)
b = FieldElement (num = 7, prime = prime)
x1 = FieldElement (num = 192, prime = prime)
y1 = FieldElement (num = 105, prime = prime)
x2 = FieldElement (num = 17, prime = prime)
y2 = FieldElement (num = 56, prime = prime)
p1 = Point (x1, y1, a, b)
p2 = Point (x2, y2, a, b)
print (p1 + p2)

None


In [69]:
prime = 223
a = FieldElement (0, prime)
b = FieldElement (7, prime)
x = FieldElement (47, prime)
y = FieldElement (71, prime)
p = Point (x, y, a, b)

for s in range (1, 21):
    result = s*p
    print ('{} * (47, 71) = ({}, {})'.format (s, result.x.num, result.y.num))

1 * (47, 71) = (47, 71)


AttributeError: 'NoneType' object has no attribute 'x'

**Coding scalar multiplication**

In [16]:
prime = 157
a = FieldElement (0, prime)
b = FieldElement (7, prime)
x = FieldElement (15, prime)
y = FieldElement (56, prime)
p = Point (x, y, a, b)
print (5*p)

TypeError: unsupported operand type(s) for +=: 'NoneType' and 'Point'

**Working with secp256k1**

In [59]:
gx = 0x79be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798
gy = 0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8
p = 2**256 - 2**32 - 977
print (gy**2 % p == (gx**3 +7) % p)

True


In [60]:
gx = 0x79be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798
gy = 0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8
p = 2**256 - 2**32 - 977
n = 0xfffffffffffffffffffffffffffffffebaaedce6af48a03bbfd25e8cd0364141

x = FieldElement (gx, p)
y = FieldElement (gy, p)

eight = FieldElement (8, p)
zero = FieldElement (0, p)

G = Point (x, y, zero, eight)

print (n * G)

ValueError: (FieldElement_115792089237316195423570985008687907853269984665640564039457584007908834671663(55066263022277343669578718895168534326250603453777594175500187360389116729240), FieldElement_115792089237316195423570985008687907853269984665640564039457584007908834671663(32670510020758816978083085130507043184471273380659243275938904335757337482424)) is not on the curve

In [62]:
class S256Field (FieldElement):
    def __init__ (self, num, prime = None):
        super ().__init__ (num = num, prime = P)
        
    def __repr__ (self):
        return '{:x}'.format (self.num).zfill (64)

In [63]:
class S256Point (Point):
    def __init__ (self, x, y, a = None, b = None):
        a, b = S256Field (a), S256Field (b)
        if type (x) == int:
            super ().__init__(x = S256Field (x), y = S256Field (y), a = a, b = b)
        else:
            super ().__init__ (x = x, y = y, a = a, b = b)
            
    def __rmul__ (self, coefficient):
        coef = coefficient % N
        return super ().__rmul__ (coef)

In [64]:
G = S256Point (0x79be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798,
              0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8)

print (N * G)

NameError: name 'P' is not defined

**Verifying a signature**

In [65]:
z = 0xbc62d4b80d9e36da29c16c5d4d9f11731f36052c72401a76c23c0fb5a9b74423
r = 0x37206a0610995c58074999cb9767b87af4c4978db68c06e8e6e81d282047a7c6
s = 0x8ca63759c1157ebeaec0d03cecca119fc9a75bf8e6d0fa65c841c8e2738cdaec
px = 0x04519fac3d910ca7e7138f7013706f619fa8f033e6ec6e09370ea38cee6a7574
py = 0x82b51eab8c27c66e26c858a079bcdf4f1ada34cec420cafc7eac1a42216fb6c4

point = S256Point (px, py)
s_inv = pow (s, N - 2, N)
u = z * s_inv % N
v = r * s_inv % N
print ((u * G + v * point).x.num == r)

NameError: name 'P' is not defined